In [1]:
#Importamos librerias
from gensim.models import word2vec
import numpy as np
import gensim

In [ ]:
archivo = open('comentarios_atencion_cliente sin_etiquetas.csv','r').readlines()

In [ ]:
archivo1 = " ".join(archivo)

In [ ]:
archivo2 = archivo1.replace(";","")

In [ ]:
archivo3 = archivo2.replace("\n","")

In [ ]:
archivo4 = archivo3.strip(" ")

In [ ]:
archivo5 = archivo4.lower()

In [ ]:
archivo6 = archivo5.split(' ')

In [ ]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
#nltk.download('stopwords')

In [ ]:
#nltk.download('punkt')

In [ ]:
stop_words = set(stopwords.words("spanish"))

In [ ]:
archivo7 = word_tokenize(archivo5)

In [ ]:
archivo8 = filter(lambda x: x not in stop_words, archivo7) #Quitamos Stopwords

In [ ]:
a = list(archivo8) #Convertimos a lista

In [ ]:
#seguimos limpiando caracteres que son menores a 3 los quitamos
c=[]
for i in a:
    if len(i)>=3:
        c.append(i)
b = " ".join(c)

In [ ]:
sentences = word2vec.Text8Corpus('text8')

In [ ]:
model = word2vec.Word2Vec(sentences, size=200,min_count=2)

In [ ]:
model.save('text8_pc4.bin')

In [ ]:
model.wv.save_word2vec_format('text8_pc4.bin', binary=True)

In [ ]:
def getFV(document):#document es una cadena de palabras
    
    words=document.split()#lista de palabras separadas
    s=np.zeros(200)
    k=0#contador
    for w in words:#para cada palabra que esta dentro de words
        if w in model.wv.vocab:#si la palabra estsa la encuentra y la suma
            s=s+model[w]
            k=k+1
    
    return s/k#retorna el promedio el vector caracteristico del documento

In [ ]:
#Probamos la funcion getFV con la lista de palabras
getFV(b)

In [ ]:
#archivo_final = open("data.txt", "w") 

In [ ]:
#Creamos un archivo que almacenará la lista de palabras limpias del conjunto de comentarios
with open("data.txt", 'w', encoding ='utf8') as f:  
            f.write(b)

In [ ]:
#Probamos que el archivo fue guardado correctamente
archivoprueba = open("data.txt", 'r').readlines()

In [ ]:
#Creamos un sentences el cual será usado en el modelo para reentrenarlo
sentences2 = word2vec.Text8Corpus("data.txt")

In [ ]:
sentences2

In [ ]:
#Anadimos el Sentences al modelo
model.build_vocab(sentences2,update=True)

In [ ]:
#Entrenamos el modelo con el nuevo sentences, pues contiene datos nuevos en ESP para poder clasificar
model.train(sentences2, total_examples=model.corpus_count, epochs=model.iter) #reentrenamos el modelo con las palabras añadidas

In [ ]:
#Verificamos que el modelo esta funcionando correctamente. Esto se hace con lo siguiente:
model.most_similar(['servicio'])

In [ ]:
#Creamos una lista que contendra lista de comentarios limpios
m = []
for i in archivo:
    a = str(i)
    b=a.replace(";","")
    c = b.replace("\n","")
    d = c.strip(' ')
    e = d.replace("  ", " ")
    f = e.lower()
    m.append(f)

In [ ]:
#Creamos una lista que contendra los arrays caracteristicos usando la funcion getFV
X=[]
for w in m:
    a=getFV(w)
    X.append(a)

In [ ]:
len(X)#Verificamos el tamaño de la lista de vectores caracteristicos

In [ ]:
#Podemos usar la siguiente operacion para poder obtener la lista limpia sin NaN
#Si cambiamos if not por if podemos obtener solo la lista de NaN
#No se uso esta operacion pues para terminos de analisis de Machine Learnin era necesario mantener las listas con
#Archivos NaN
#import math
#res2 = [t for t in X if not any(isinstance(n, float) and math.isnan(n) for n in t)]

In [ ]:
#len(res2)

In [ ]:
#Limpiamos la lista de vectores caracteristicos pues hay datos que al no encontralos los vuelve vectores NaN
#Esto lo hacemos con
b = np.where(np.isnan(X), 0, X)
#aqui encuentra los nan y los reemplaza por ceros. Cuestion que no sesga mucho pues solo hemos identificado que son 20

In [ ]:
#Verificamos que la nueva lista limpia tiene el mismo tamaño
len(b)

In [ ]:
len(X)#Verificamos nuevamente el tamaño de las listas

In [ ]:
#Importamos la libreria de KMeans 
from sklearn.cluster import KMeans

In [ ]:
#Hacemos KMeans de 2 K para b
est = KMeans(2)
est.fit (b)
y_mean=est.predict(b)

In [ ]:
y_mean#es el modelo

In [ ]:
#Abrimos el archivo de positivos para verificar el Accuracy
archivoPOSI = open('Positive.csv','r').readlines()

In [ ]:
#Creamos una lista de los comentarios positivos
POSI = []
for i in archivoPOSI:
    g = str(i)
    h = g.replace(";","")
    i = h.replace("\n","")
    j = i.strip(' ')
    k = j.replace("  ", " ")
    l = k.lower()
    POSI.append(l)

In [ ]:
#Usando getFV appendamos los vectores caracteristicos en una lista P de positivos
P=[]
t=0
for w in POSI:
    a=getFV(w)
    P.append(a)

In [ ]:
#Limpiamos la lista y convertimos los NaN en Ceros
p_ = np.where(np.isnan(P), 0, P)

In [ ]:
#Hacemos KMeans para la lista p_ la cual contiene los vectores caracteristicos de los comentarios positivos
estp = KMeans(2)
estp.fit (p_)
y_meanp=estp.predict(p_)

In [ ]:
y_meanp # POSITIVOS K MEANS

In [ ]:
#Hallamos el Accuracy de los positivos usando el algoritmo KMeans
upk=0
for i in y_meanp:
    if i == 1:
        upk=upk+1
print (upk/(len(y_meanp)))

In [ ]:
#Abrimos el archivo de negativos para verificar el Accuracy
archivoNEGA = open('Negative.csv','r').readlines()

In [ ]:
#Creamos una lista de los comentarios positivos y la limpiamos
NEGA = []
for i in archivoNEGA:
    g = str(i)
    h = g.replace(";","")
    i = h.replace("\n","")
    j = i.strip(' ')
    k = j.replace("  ", " ")
    l = k.lower()
    NEGA.append(l)

In [ ]:
#Usando getFV appendamos los vectores caracteristicos en una lista P de negativos
N=[]
for w in NEGA:
    a=getFV(w)
    N.append(a)

In [ ]:
#Limpiamos la lista n y convertimos los NaN en Ceros
n_ = np.where(np.isnan(N), 0, N)

In [ ]:
#Hallamos el Accuracy de los negativos usando el algoritmo KMeans
estn = KMeans(2)
estn.fit (n_)
y_meann=estn.predict(n_)

In [ ]:
y_meann #NEGATIVOS K MEANS

In [ ]:
#Usando getFV appendamos los vectores caracteristicos en una lista P de negativos
unk=0
for i in y_meann:
    if i == 1:
        unk=unk+1
print (unk/(len(y_meann)))

In [ ]:
#Abrimos el csv etiquetado
archivoSVM = open('label-comments_1400.csv','r').readlines()

In [ ]:
#Importamos la libreria pandas. Creamos el dataframe
import pandas as pd
df = pd.read_csv('label-comments_1400.csv', delimiter = ';', header=None)

In [ ]:
#Creamos data frame con solo las columnas necesarias solo para ser ordenados
df2 = df.drop(columns=[2,3,4,5])

In [ ]:
etiquetas = df2[0]#data frame de solo las etiquetas
comments = df2[1]#data frame de solo los comentarios

In [ ]:
#Creamos la lista para limpiarla
c_c = []
for i in comments:
    g = str(i)
    h = g.replace(";","")
    i = h.replace("\n","")
    j = i.strip(' ')
    k = j.replace("  ", " ")
    l = k.lower()
    c_c.append(l)

In [ ]:
#Creamos la lista que contendra los vectores caracteristicos de todos los comentarios
co_v=[]
for w in c_c:
    a=getFV(w)
    co_v.append(a)

In [ ]:
#Limpiamos la lista de los NaN y lo convertimos en ceros
d=np.where(np.isnan(co_v), 0, co_v)

In [ ]:
#Algoritmo de Support Vector Machine
from sklearn import svm
clasificador = svm.SVC(gamma=0.00001,C=10)

In [ ]:
#Ejecutamos el clasificador SVM donde d es la lista de FV de los comentarios y etiquetas, las etiquetas
clasificador.fit(d,etiquetas)

In [ ]:
Y_predNEGAS = clasificador.predict(n_) #NEGATIVOS SVM

In [ ]:
Y_predNEGAS #Imprimimos el array de negativos con SVM

In [ ]:
#Hallamos el Accuracy de los negativos con SVM
un=0
for i in Y_predNEGAS:
    if i == 1:
        un=un+1
print (un/(len(Y_predNEGAS)))

In [ ]:
Y_predPOSI = clasificador.predict(p_) #POSITIVOS SVM

In [ ]:
Y_predPOSI #Imprimimos el array de Positivos con SVM

In [ ]:
#Hallamos el Accuracy de los negativos con SVM
up=0
for i in Y_predPOSI:
    if i == 1:
        up=up+1
print (up/(len(Y_predPOSI)))